PROCESO ML

#### Bibliotecas

In [1]:
import bootcampviztools as bt
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

# API MARVEL

In [2]:
# Claves de la API
public_key = '0399714820d7f8388e74266a3c94cb4a'
private_key = '7fe33c6de400b6bafdd3ba7f8b3158681c092aed'

In [4]:
# Parametros de autenticacion
def get_hash_params(public_key, private_key):
    ts = str(time.time()) 
    hash_string = ts + private_key+ public_key 
    hash_result = hashlib.md5(hash_string.encode()).hexdigest()
    return {
        'ts':ts,
        'apikey': public_key,
        'hash': hash_result,
    }

In [5]:
# Funcion obtener todos los personajes
def fetch_all_characters():
        base_url = 'http://gateway.marvel.com/v1/public/characters'
        all_characters = []
        limit = 100
        offset = 0

        while True:
                params = get_hash_params(public_key, private_key)
                params.update({
                        'limit': limit,
                        'offset': offset
                })

                response = requests.get(base_url, params = params)

                if response.status_code != 200:
                        print(f'Error: {response.status_code}')
                        break
                
                data = response.json()
                characters = data['data']['results']

                if not characters:
                        break

                all_characters.extend(characters)
                ofset += limit

                # Mostrar progreso
                print(f"Obtenidos {len(all_characters)} personajes...")
                
                # Pausa para respetar los limite de la API
                time.sleep(1)
        
        return all_characters

In [6]:
# Funcion convertir datos en un dataframe
def characters_to_dataframe(characters):
    character_data = []
    for character in characters:
        character_data.append({
            'id': character['id'],
            'name': character['name'],
            'description': character['description'],
            'modified': character['modified'], 
            'comics_available': character['comics']['available'],
            'series_available': character['series']['available'],
            'stories_available': character['stories']['available'],
            'events_available': character['stories']['available'],
            'event_available': character['events']['available'], 
            'thumbnail': f"{character['thumbnail']['path']}.{character['thumbnail']['extension']}"
        })
    df = pd.DataFrame(character_data)
    return df

In [14]:
# Obtener personajes
all_characters = fetch_all_characters()

# Covnertir datos en un Dataframe
characters_df = characters_to_dataframe(all_characters)
characters_df.head()

Obtenidos 100 personajes...
Obtenidos 200 personajes...
Obtenidos 300 personajes...
Obtenidos 400 personajes...
Obtenidos 500 personajes...
Obtenidos 600 personajes...
Obtenidos 700 personajes...
Obtenidos 800 personajes...
Obtenidos 900 personajes...
Obtenidos 1000 personajes...
Obtenidos 1100 personajes...
Obtenidos 1200 personajes...
Obtenidos 1300 personajes...
Obtenidos 1400 personajes...
Obtenidos 1500 personajes...
Obtenidos 1564 personajes...


,id,name,description,modified,comics_available,series_available,stories_available,events_available,event_available,thumbnail
0,1011334,3-D Man,,2014-04-29T14:18:17-0400,12,3,21,21,1,http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04-0400,4,2,7,7,0,http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30-0400,53,36,57,57,0,http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...
3,1010699,Aaron Stack,,1969-12-31T19:00:00-0500,14,3,27,27,0,http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2012-03-20T12:32:12-0400,58,28,68,68,1,http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...


In [16]:
characters_df.to_csv('Marvel_characters.csv', index = False)

In [17]:
characters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1564 non-null   int64 
 1   name               1564 non-null   object
 2   description        1564 non-null   object
 3   modified           1564 non-null   object
 4   comics_available   1564 non-null   int64 
 5   series_available   1564 non-null   int64 
 6   stories_available  1564 non-null   int64 
 7   events_available   1564 non-null   int64 
 8   event_available    1564 non-null   int64 
 9   thumbnail          1564 non-null   object
dtypes: int64(6), object(4)
memory usage: 122.3+ KB


# WEB SCRAPING SUPERHEROES AND VILLAINS

In [2]:
import lxml
import requests
import pandas as pd
from bs4 import BeautifulSoup

ALFRED

In [35]:
url = 'https://www.superherodb.com/alfred/10-14275/'
response = requests.get(url)
response 

<Response [200]>

In [7]:
# from fake_useragent import UserAgent                #ya que me niega el acceso, hay que ejecutar este código
# ua = UserAgent()
# headers = {'User-Agent': ua.random}
# response = requests.get(url, headers = headers)
# response

<Response [200]>

In [36]:
# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'column col-10 col-md-9 col-sm-12'
    div_column = soup.find('div', class_='column col-10 col-md-9 col-sm-12')
    
    # Extraer el nombre completo del superhéroe (h2 dentro del div encontrado)
    nombre_completo = div_column.find('h2').text.strip()
    
    # Extraer el nombre del superhéroe (primer h1 dentro del div encontrado)
    nombre = div_column.find('h1').text.strip()
    
    # Extraer el universo del superhéroe (primer h3 dentro del div encontrado)
    universo = div_column.find('h3').text.strip()
    
    # Imprimir los resultados
    print(f"Nombre completo: {nombre_completo}")
    print(f"Nombre: {nombre}")
    print(f"Universo: {universo}")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


Nombre completo: Alfred Pennyworth
Nombre: Alfred
Universo: DC Extended Universe


In [25]:
# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'stat-holder stat-rows'
    stat_holder = soup.find('div', class_='stat-holder stat-rows')
    
    if stat_holder:
        # Encontrar todos los elementos div con clase 'stat-bar' dentro de 'stat-holder'
        stat_bars = stat_holder.find_all('div', class_='stat-bar')
        
        # Iterar sobre cada stat-bar para extraer los stats y stat-values
        for stat_bar in stat_bars:
            # Encontrar el stat-value dentro de cada stat-bar
            stat_value = stat_bar.find('div', class_='stat-value')
            if stat_value:
                stat_name = stat_bar.find('label').text.strip()
                stat_value_text = stat_value.text.strip()
                print(f"{stat_name}: {stat_value_text}")
            else:
                print("No se encontró el stat-value dentro de stat-bar")
    else:
        print("No se encontró el elemento stat-holder stat-rows en la página.")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


Intelligence: 0
Strength: 0
Speed: 0
Durability: 0
Power: 0
Combat: 0
Tier: 0
Tier2: 0


AMANDA WALLER 

In [34]:
url = 'https://www.superherodb.com/amanda-waller/10-33322/'
response = requests.get(url)
response 

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'column col-10 col-md-9 col-sm-12'
    div_column = soup.find('div', class_='column col-10 col-md-9 col-sm-12')
    
    # Extraer el nombre completo del superhéroe (h2 dentro del div encontrado)
    nombre_completo = div_column.find('h2').text.strip()
    
    # Extraer el nombre del superhéroe (primer h1 dentro del div encontrado)
    nombre = div_column.find('h1').text.strip()
    
    # Extraer el universo del superhéroe (primer h3 dentro del div encontrado)
    universo = div_column.find('h3').text.strip()
    
    # Imprimir los resultados
    print(f"Nombre completo: {nombre_completo}")
    print(f"Nombre: {nombre}")
    print(f"Universo: {universo}")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Encontrar el elemento div con clase 'stat-holder stat-rows'
    stat_holder = soup.find('div', class_='stat-holder stat-rows')
    
    if stat_holder:
        # Encontrar todos los elementos div con clase 'stat-bar' dentro de 'stat-holder'
        stat_bars = stat_holder.find_all('div', class_='stat-bar')
        
        # Iterar sobre cada stat-bar para extraer los stats y stat-values
        for stat_bar in stat_bars:
            # Encontrar el stat-value dentro de cada stat-bar
            stat_value = stat_bar.find('div', class_='stat-value')
            if stat_value:
                stat_name = stat_bar.find('label').text.strip()
                stat_value_text = stat_value.text.strip()
                print(f"{stat_name}: {stat_value_text}")
            else:
                print("No se encontró el stat-value dentro de stat-bar")
    else:
        print("No se encontró el elemento stat-holder stat-rows en la página.")

else:
    print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")


Nombre completo: Amanda Blake Waller
Nombre: Amanda Waller
Universo: DC Extended Universe
Intelligence: 0
Strength: 0
Speed: 0
Durability: 0
Power: 0
Combat: 0
Tier: 0
Tier2: 0


# DC EXTENDED UNIVERSE 

In [ ]:
# Diccionario 
dc_extended_universe = {
    'Adrianna Tomaz': 'https://www.superherodb.com/alfred/10-14275/#shdb',
    'Amanda Waller': 'https://www.superherodb.com/alfred/10-14275/#shdb',
    'Anthea': 'https://www.superherodb.com/anthea/10-54768/#user',
    'Aquaman': 'https://www.superherodb.com/aquaman/10-12533/',
    'Ares': 'https://www.superherodb.com/ares/10-12597/',
    'Atom Smasher': 'https://www.superherodb.com/atom-smasher/10-27865/',
    'Batman': 'https://www.superherodb.com/batman/10-12530/',
    'Black Adam': 'https://www.superherodb.com/black-adam/10-21003/',
    'Black Canary': 'https://www.superherodb.com/black-canary/10-18617/',
    
}